## Part 1g: TensorFlow - Custom Dropout and Custom Regularization

**Description:**

This Colab demonstrates how to implement custom dropout behavior and custom regularization techniques in TensorFlow Keras. While Keras provides built-in layers for standard dropout and regularizers like L1 and L2, you might have specific requirements that necessitate custom implementations.

**Custom Dropout:** We will create a custom layer that implements a specific dropout pattern (e.g., dropping out entire feature maps instead of individual neurons, although for simplicity we'll stick to a similar neuron-wise dropout but as a custom layer).

**Custom Regularization:** We will define a custom regularizer function that applies a specific penalty to the weights of a layer, different from the standard L1 or L2. For this example, we will implement a custom L1 regularizer with a potentially different scaling mechanism.

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

# Load the digits dataset
digits = load_digits()
X, y = digits.data, digits.target

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 1. Custom Dropout Layer - Corrected to use TensorFlow random ops
class CustomDropout(Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = min(1., max(0., rate))
        self.supports_masking = True

    def call(self, inputs, training=None):
        if training is None or not training:
            return inputs
        keep_prob = 1. - self.rate
        random_tensor = tf.random.uniform(shape=tf.shape(inputs))
        mask = tf.cast(random_tensor >= self.rate, dtype=inputs.dtype)
        return inputs * mask / keep_prob

    def get_config(self):
        config = super().get_config()
        config.update({'rate': self.rate})
        return config

# 2. Custom Regularizer (Modified L1) - No changes needed here
class CustomL1Regularizer(regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, weight_matrix):
        return self.factor * K.sum(K.abs(weight_matrix)) ** 2  # Example: L1 penalty squared

    def get_config(self):
        return {'factor': self.factor}

# Create models using custom dropout and regularization
model_custom = models.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer=CustomL1Regularizer(0.001), input_shape=(X_train.shape[1],)),
    CustomDropout(0.5),
    layers.Dense(10, activation='softmax')
])

model_custom.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_custom = model_custom.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=0)
evaluation_custom = model_custom.evaluate(X_test, y_test, verbose=0)
loss_custom = evaluation_custom[0]
accuracy_custom = evaluation_custom[1]
print(f"Model with Custom Dropout and Regularization - Test Accuracy: {accuracy_custom}")

# Compare with a model using standard Dropout and L1 regularization
model_standard = models.Sequential([
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001), input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model_standard.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history_standard = model_standard.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=0)
evaluation_standard = model_standard.evaluate(X_test, y_test, verbose=0)
loss_standard = evaluation_standard[0]
accuracy_standard = evaluation_standard[1]
print(f"Model with Standard Dropout and L1 Regularization - Test Accuracy: {accuracy_standard}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model with Custom Dropout and Regularization - Test Accuracy: 0.9305555820465088
Model with Standard Dropout and L1 Regularization - Test Accuracy: 0.980555534362793


## Results for Part 1g: TensorFlow - Custom Dropout and Custom Regularization

In this experiment, we implemented a custom dropout layer (`CustomDropout`) and a custom L1 regularizer (`CustomL1Regularizer`) in TensorFlow Keras. We then trained a simple neural network using these custom components and compared its performance to a network using standard Keras `Dropout` and `L1` regularization. Both models were trained on the `digits` dataset for 50 epochs.

The test accuracies achieved by each model are as follows:

* **Model with Custom Dropout and Regularization - Test Accuracy:** 0.9306
* **Model with Standard Dropout and L1 Regularization - Test Accuracy:** 0.9806

**Analysis:**

The results indicate a noticeable difference in performance between the two models:

* The model utilizing the **standard Keras Dropout and L1 regularization** achieved a significantly higher test accuracy of 0.9806 compared to the 0.9306 achieved by the model with the custom implementations.

This suggests that, for this specific task and the chosen hyperparameters:

* The standard Keras `Dropout` layer and `L1` regularizer were more effective in promoting generalization.
* Our custom `CustomDropout` implementation, while functionally similar to standard dropout, might have subtle differences in its interaction with the rest of the network or the optimization process that led to lower performance.
* The custom `CustomL1Regularizer`, which applied a squared L1 norm penalty, also appears to be less effective than the standard L1 regularizer (which applies the absolute value penalty) in this scenario.

**A/B Test Comparison:**

This experiment highlights the importance of carefully implementing and testing custom layers and regularizers. While Keras provides flexibility for customization, the built-in components are often well-optimized and have been proven effective across a wide range of tasks. In this case, the standard Keras regularization and dropout outperformed our custom versions. Further investigation and potential refinement of the custom implementations would be needed to understand why they underperformed.